In [219]:
import psycopg2

In [220]:
conn = psycopg2.connect("host=127.0.0.1 dbname=postgres user=postgres password=postgres")
cur = conn.cursor()

In [221]:
cur.execute("SELECT * FROM user")

In [222]:
cur.fetchall()

[('postgres',)]

In [223]:
cur.execute("CREATE Table test_02 (id serial PRIMARY KEY, imie varchar, nazwisko varchar, wiek integer);")

ProgrammingError: relation "test_02" already exists


In [ ]:
#cur.execute("DROP Table test_01")

In [ ]:
cur.execute("INSERT INTO test_02 (imie, nazwisko, wiek) values('Tadeusz', 'Piatkowski', 0);")

In [ ]:
cur.execute("INSERT INTO test_02 (imie, nazwisko, wiek) values('Kasia', 'Piatkowska', 31);")

In [ ]:
cur.execute("INSERT INTO test_02 (imie, nazwisko, wiek) values('Lukasz', 'Piatkowski', 35);")

In [ ]:
conn.commit()

In [ ]:
cur.execute("SELECT * FROM test_01")

In [ ]:
cur.fetchall()

In [ ]:
#conn.rollback()

In [ ]:
z = cur.fetchall()

In [ ]:
type(z)

In [ ]:
import sqlalchemy
sqlalchemy.__version__

# Tworzenei silnika

In [ ]:
from sqlalchemy import create_engine
engine_file = create_engine('sqlite:///:testowo.sqlite)', echo=True)

# Deklaracja mapingu

In [ ]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

# Tworzenie schematu i sesji

In [ ]:
from sqlalchemy import Column, Integer, String
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine_file)
session = Session()

# Tworzenie klasy

In [ ]:
class Test1(Base):
    __tablename__ = 'lukifam'
    
    id = Column(Integer, primary_key=True)
    name = Column(String(30), nullable=False)
    surename = Column(String)
    age = Column (Integer)
    birth_d = Column (String)
    father = Column (String)
    
    def __repr__(self):
        return "<User(name='%s', surename='%s', age='%s', birth_d='%s',father='%s')>" % (
            self.name, self.surename, self.age, self.birth_d, self.father)

In [ ]:
Base.metadata.create_all(engine_file)

In [ ]:
Tadek = Test1(name="Tadeusz", surename="Piatkowski", age=0, birth_d="18-08-2018", father="Lukasz Piatkowski")
Kasia = Test1(name="Katarzyna", surename="Piatkowska", age=31, birth_d="27-12-1987", father="Slawomir Romaniuk")
Luki = Test1(name="Lukasz", surename="Piatkowski", age=35, birth_d="12-12-1982", father="Tadeusz Piatkowski")
Tadek_dziadek = Test1(name="Tadeusz", surename="Piatkowski", age=68, birth_d="01-01-1950", father="Andrzej Piatkowski")
Zofia = Test1(name="Zofia", surename="Piatkowska", age=66, birth_d="20-01-2018", father="Tadeusz Koziol")
Ania = Test1(name="Anna", surename="Wyroslak", age=0, birth_d="01-01-1987", father="Tadeusz Piatkowski")

In [ ]:
session.add_all([Tadek, Kasia, Luki, Tadek_dziadek, Zofia, Ania])

In [ ]:
session.commit()

In [ ]:
Aga = Test1(name="Agnieszka", surename="Shanks", age=42, birth_d="10-10-1976", father="Tadeusz Piatkowski")

In [ ]:
session.new

In [ ]:
Aga in session

In [ ]:
session.deleted

In [ ]:
session.add_all([Aga])

In [ ]:
session.commit()

# Zapytania

In [ ]:
session.query(Test1).all()

In [ ]:
session.query(Test1).order_by(Test1.age).all()

In [ ]:
Ania.age = 31

In [ ]:
session.query(Test1).order_by(Test1.age).all()

In [ ]:
session.query(Test1).filter(Test1.name.in_(['Tadeusz', 'Lukasz'])).all()

In [ ]:
from sqlalchemy import desc, asc

In [ ]:
session.query(Test1).order_by(desc(Test1.age)).all()

In [ ]:
for ppl in session.query(Test1).order_by(desc(Test1.age))[1:4]:
    print(ppl)

In [ ]:
session.query(Test1).filter(Test1.name=='Tadeusz').filter(Test1.father=='Lukasz Piatkowski').all()

In [ ]:
session.query(Test1).filter(Test1.name.ilike('%Tadeusz%')).all()

In [ ]:
session.query(Test1).filter(Test1.name.like('%Tadeusz%')).all()

In [ ]:
from sqlalchemy import and_
from sqlalchemy import or_

In [ ]:
session.query(Test1).filter(or_(Test1.name=='Tadeusz',Test1.father=='Lukasz Piatkowski'))

In [ ]:
q = session.query(Test1).from_statement(
    sqlalchemy.text("SELECT * FROM lukifam WHERE name=:name_param")).params(name_param = "Tadeusz")
print(q.all())

In [ ]:
session.query(Test1).filter(Test1.name=='Tadeusz').count()

# Budowanie relacji

In [ ]:
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship

In [ ]:
session.new

class Adress(Base):
    __tablename__ = 'adresyl'
    __table_args__ = {'extend_existing': True}
    id = Column(Integer, primary_key=True)
    adres_zam = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey('lukifam.id'))
    lukiad = relationship("Test1", back_populates="adr")
    
    def __repr__(self):
        return "< Adres zamieszkania: (adres_zam='%s') >" % self.adres_zam

In [232]:
class Test3(Base):
    __tablename__ = 'lukifam1'
    __table_args__ = {'extend_existing': True}
    id = Column(Integer, primary_key=True)
    name = Column(String(30), nullable=False)
    surename = Column(String)
    age = Column (Integer)
    birth_d = Column (String)
    father = Column (String)
    adr = relationship("Adress2", order_by=Adress2.id, back_populates="lukiad")
    
    def __repr__(self):
        return "<User(name='%s', surename='%s', age='%s', birth_d='%s',father='%s')>" % (
            self.name, self.surename, self.age, self.birth_d, self.father)
    
class Adress2(Base):
    __tablename__ = 'adresyl1'
    __table_args__ = {'extend_existing': True}
    id = Column(Integer, primary_key=True)
    adres_zam = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey('lukifam1.id'))
    lukiad = relationship("Test3", back_populates="adr")
    
    def __repr__(self):
        return "< Adres zamieszkania: (adres_zam='%s') >" % self.adres_zam
    
    

NameError: name 'Adress2' is not defined

In [231]:
Base.metadata.create_all(engine_file)

2018-11-04 19:29:38,730 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("lukifam")
2018-11-04 19:29:38,747 INFO sqlalchemy.engine.base.Engine ()
2018-11-04 19:29:38,760 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("parkinglot1")
2018-11-04 19:29:38,765 INFO sqlalchemy.engine.base.Engine ()
2018-11-04 19:29:38,777 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("car1")
2018-11-04 19:29:38,782 INFO sqlalchemy.engine.base.Engine ()
2018-11-04 19:29:38,789 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("adresyl")
2018-11-04 19:29:38,793 INFO sqlalchemy.engine.base.Engine ()
2018-11-04 19:29:38,798 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE adresyl (
	id INTEGER NOT NULL, 
	adres_zam VARCHAR NOT NULL, 
	user_id INTEGER, 
	PRIMARY KEY (id), 
	FOREIGN KEY(user_id) REFERENCES lukifam (id)
)


2018-11-04 19:29:38,803 INFO sqlalchemy.engine.base.Engine ()
2018-11-04 19:29:43,900 INFO sqlalchemy.engine.base.Engine ROLLBACK


OperationalError: (sqlite3.OperationalError) database is locked [SQL: '\nCREATE TABLE adresyl (\n\tid INTEGER NOT NULL, \n\tadres_zam VARCHAR NOT NULL, \n\tuser_id INTEGER, \n\tPRIMARY KEY (id), \n\tFOREIGN KEY(user_id) REFERENCES lukifam (id)\n)\n\n']

In [227]:
adres = Adress1(adres_zam='Nowogrodzka 62')

InvalidRequestError: Multiple classes found for path "Test1" in the registry of this declarative base. Please use a fully module-qualified path.

In [ ]:
repr(Ania)

In [ ]:
Ania.adresyl = ('ul. Relaksowa 12 m.4, 04-511 Lublin')

In [ ]:
Ania.adresyl.lukifam

Tadek.adresyl = Adresy(adres_zam='ul. Jana Kazimierza 53, 01-267 Warszawa')
Kasia.adresyl = Adresy(adres_zam='ul. Jana Kazimierza 53, 01-267 Warszawa')
Luki.adresyl = Adresy(adres_zam='ul. Jana Kazimierza 53, 01-267 Warszawa')
Tadek_dziadek.adresyl = Adresy(adres_zam='ul. Mlynarska 20, 08-110 Siedlce')
Zofia.adresyl = Adresy(adres_zam='ul. Mlynarska 20, 08-110 Siedlce')
Aga.adresyl = Adresy(adres_zam='Bulmore str. 12, 004116 New Jersy')

In [ ]:
#session.deleted

In [ ]:
class ParkingLot1(Base):
    __tablename__ = 'parkinglot1'
    __table_args__ = {'extend_existing': True}
    id = Column(Integer, primary_key=True)
    address = Column(String)
    cars = relationship("Car1", back_populates="parking_lot1")

class Car1(Base):
    __tablename__ = 'car1'
    __table_args__ = {'extend_existing': True}
    id = Column(Integer, primary_key=True)
    brand = Column(String)
    model = Column(String)
    color = Column(String)
    parking_lot_id = Column(Integer, ForeignKey('parkinglot1.id'))
    parking_lot1 = relationship("ParkingLot1", back_populates="cars")

In [ ]:
Base.metadata.create_all(engine_file)

In [ ]:
parking = ParkingLot1(address='Nowogrodzka 62')

In [ ]:
car = Car1(brand='audi', model='80', color='red', parking_lot1=parking)

In [ ]:
session.add_all([parking, car])

In [ ]:
car.parking_lot_id

In [ ]:
session.query(Car1).all()